# Scan for one port

In [1]:
package main

import (
    "fmt"
    "net"
)

func main() {
    _, err := net.Dial("tcp", "scanme.nmap.org:80")
    if err == nil {
        fmt.Println("Connection successful")
    }
}

main()

Connection successful


# performing noconcurrent scanning

TCP ports range from 1 to 65535

In [2]:
for i:=1; i <= 18; i++ {
    address := fmt.Sprintf("scanme.nmap.org:%d", i)
    connection, err := net.Dial("tcp", address)
    if err != nil {
        // port is closed or filtered
        continue
    }
    connection.Close()
    fmt.Printf("%d is open\n", i)
}

1 is open
2 is open
3 is open
4 is open
5 is open
6 is open
7 is open
8 is open
9 is open
10 is open
11 is open
12 is open
13 is open
14 is open
15 is open
16 is open
17 is open
18 is open


# Performing noconcurrent scanning

In [3]:
package main 

import (
    "fmt"
    "net"
    "sync"
)

func main() {
    var waitGroup sync.WaitGroup
    for i:=1; i <= 65535; i++ {
        waitGroup.Add(1)
        go func(j int) {
            defer waitGroup.Done() //we call this function when this function is done(). wg.Done() will do this: waitGroup_counting -= 1
            address := fmt.Sprintf("localhost:%d", j)
            connection, err := net.Dial("tcp", address)
            if err != nil {
                // port is closed or filtered
                return
            }
            connection.Close()
            fmt.Printf("%d is open\n", j)
        }(i)
    }
    waitGroup.Wait()
}

main()

443 is open
22 is open
631 is open
1716 is open
5900 is open
6800 is open
6379 is open
5000 is open
8889 is open
8888 is open
26552 is open
33657 is open
43110 is open
43553 is open
51413 is open
63342 is open


In this version of the program, you create sync.WaitGroup u, which acts as a synchronized counter. 

You increment this counter via wg.Add(1) each time you create a goroutine to scan a port v, and a deferred call to wg.Done() decrements the counter whenever one unit of work has been performed w.

Your main() function calls wg.Wait() , which blocks until all the work has been done and your counter has returned to zero x.

**`cap()` tells you the capacity of the underlying array. len tells you how many items are in the array. **

# Port Scanning Using a Worker Pool

In [4]:
package main 

import (
    "fmt"
    "sync"
)

func worker(ports chan int, wg *sync.WaitGroup) {
    for p := range ports{
        fmt.Println(p)
        wg.Done()
    }
}

func main() {
    ports := make(chan int, 100)
    var wg sync.WaitGroup
    for i:=0; i < cap(ports); i++ {
        go worker(ports, &wg)
    }
    for i := 1; i <= 1024; i++ {
        wg.Add(1)
        ports <- i 
    }
    wg.Wait()
    close(ports)
}

First, you create a channel by using make() v. A second parameter, an int value of 100 , is provided to make() here. This allows the channel to be buffered, which means you can send it an item without waiting for a receiver to read the item. Buffered channels are ideal for maintaining and track- ing work for multiple producers and consumers. You’ve capped the chan- nel at 100, meaning it can hold 100 items before the sender will block. This is a slight performance increase, as it will allow all the workers to
start immediately.

Next, you use a for loop w to start the desired number of workers—in this case, 100. In the worker(int, *sync.WaitGroup) function, you use range u to continuously receive from the ports channel, looping until the channel is closed. Notice that you aren’t doing any work yet in the worker—that’ll come shortly. Iterating over the ports sequentially in the main() function, you send a port on the ports channel x to the worker. After all the work has been completed, you close the channel y.

Once you build and execute this program, you’ll see your numbers printed to the screen. You might notice something interesting here: the numbers are printed in no particular order. Welcome to the wonderful world of parallelism.

# Multichannel Version

In [5]:
package main

import (
    "fmt"
    "net"
    "sort"
)

func worker(ports, results chan int) {
    for p := range ports {
        address := fmt.Sprintf("localhost:%d", p)
        connection, err := net.Dial("tcp", address)
        if err != nil {
            results <- 0
            continue
        }
        connection.Close()
        results <- p
    }
}

func main() {
    ports := make(chan int, 10000)
    results := make(chan int)
    var openPorts []int
    
    for i:=0; i < cap(ports); i++ {
        go worker(ports, results)
    }
    
    go func() {
        for i:=1; i <= 65535; i++ {
            ports <- i
        }
    }()
    
    for i:=0; i < 65535; i++ {
        port := <-results
        if port != 0 {
            openPorts = append(openPorts, port)
        }
    }
    
    close(ports)
    close(results)
    sort.Ints(openPorts)
    for _, port := range openPorts {
        fmt.Printf("%d open\n", port)
    }
}

main()

443 open
6379 open
6800 open
6942 open
8000 open
8080 open
8888 open
8889 open
9091 open
26552 open
45911 open
56257 open
63342 open
